In [29]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
# Step 1: Load vector store
vector_store = Chroma(
    collection_name='my_collection',
    persist_directory='chroma_db',
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

In [ ]:
# vector_store._collection.get()['ids']

['250679af-5737-4b5d-a640-2cf8f6114bdc',
 'c13ae70d-c83b-47d3-b3ca-74caae23513b',
 '43568d11-9e06-4196-a259-c0ecfaf58cad',
 '3ce67330-dfc7-4c4f-9ba6-5d32470b391b',
 '0a6ebc84-a31d-41a2-b1b5-794e14bc77cd',
 '3955117f-d407-4ec6-babd-cdcf57199b36',
 '2d5cf9c1-69a4-48a5-8974-581c57a8d35e',
 '740d7be7-40f0-425d-8e12-3b5d16a955d0',
 '86f146da-7906-4dc8-b6d9-efc90a773203',
 'e90c937d-8c3c-4545-a28c-cdf281fd6367',
 '14c3ba00-c99b-4389-8e7c-8d27604f7d3c',
 'c4a11711-a8e4-48fb-a894-73971c2ed322',
 'e0eaea05-45c7-4fec-a070-b3bf7aaefd18',
 '7d94df79-738e-4463-accc-e7e24bd3c367',
 '438b1eb6-d90c-438d-b946-baa55c244978',
 'f48fa34d-2f99-4d47-8c1a-218fd7cbbddc',
 '51ec42d8-6ec6-448b-a2df-734582b75b8c',
 '8282b293-dc09-45a6-a849-375b9a3c0160',
 '364a9eed-022f-4a92-b60d-aeb480fd7886',
 'cee1b66b-33e8-4ddc-829e-0a8a81f0f330',
 '4d4b4a03-fbbe-4182-b79b-fab459249d08',
 '6f632a08-2678-46bb-bd46-7fb94d3fb71a',
 '4ebdde11-92e6-4f58-a987-c8b28a841704',
 '2b850a93-221f-4f24-91cd-f69d2b99d5f9',
 '71ea1415-7b1c-

In [ ]:
# Step 2: Create retriver

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [ ]:
# responses = retriever.invoke('what is allergen?')

# for response in responses:
#     print(response.page_content)
#     print('-'*5)

ins in the body due to elimination and cleansing.
Research and general acceptance
Although environmental medicine is gaining more
respect within conventional medicine, detoxification
KEY TERMS
Allergen —A foreign substance, such as mites in
house dust or animal dander, that when
inhaled,causes the airways to narrow and pro-
duces symptoms of asthma.
Antibody —A protein, also called immunoglobu-
lin, produced by immune system cells to remove
antigens (the foreign substances that trigger the
-----
8th ed. St. Louis: Mosby, 1996.
ORGANIZATIONS
American Academy of Ophthalmology. 655 Beach Street, PO
Box 7424, San Francisco, CA 94120-7424. <http://www.
eyenet.org>.
KEY TERMS
Allergen —A substance capable of inducing an
allergic response.
Allergic reaction—An immune system reaction to
a substance in the environment; symptoms
include rash, inflammation, sneezing, itchy watery
eyes, and runny nose.
Conjunctiva—The mucous membrane that covers
the white part of the eyes and lines the eyelids.
--

In [ ]:
# Step 3: Setup LLM 

model = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.5, max_output_tokens=512)

In [ ]:
# Step 4: Augmentation

def format_docs(retrived_docs):

    context_text = '\n\n'.join(doc.page_content for doc in retrived_docs)
    return context_text

In [43]:
prompt = PromptTemplate(
    template="""
    You are a helpful assistant.
    Answer ONLY from the provided context in detail.
    If the context is insufficient, just say you don't know.

    {context}
    Question: {question}
    """,
    input_variables=['context', 'question']
)

In [44]:
parser = StrOutputParser()

In [51]:
# Step 4: Create chain

parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

final_chain = parallel_chain | prompt | model | parser

In [54]:
# Step 5: Generation
user_query = input("Write query here: ")
response = final_chain.invoke(user_query)
response

'Based on the context, botulism is described as the deadliest of the bacterial food-borne illnesses.'